In [1]:
import os
import numpy as np
from PIL import Image
import pickle
from collections import Counter
import heapq
import time 
import csv

In [2]:
def compress_ecg_image(image_path):
    image = Image.open(image_path).convert('L')
    image_array = np.array(image)
    
    # --- Start Real CALIC ---
    height, width = image_array.shape
    compressed_data = []
    
    for i in range(height):
        for j in range(width):
            if i == 0 or j == 0:
                prediction = 0
            else:
                left = image_array[i, j-1]
                top = image_array[i-1, j]
                prediction = (left + top) // 2
            
            error = int(image_array[i, j]) - prediction
            compressed_data.append(error)
    # --- End Real CALIC ---
    
    compressed_array = np.array(compressed_data, dtype=np.int16)
    
    return compressed_array, image_array.shape

def decompress_ecg_image(compressed_array, shape):
    height, width = shape
    image_array = np.zeros((height, width), dtype=np.uint8)
    idx = 0
    
    for i in range(height):
        for j in range(width):
            if i == 0 or j == 0:
                prediction = 0
            else:
                left = image_array[i, j-1]
                top = image_array[i-1, j]
                prediction = (left + top) // 2
            
            pixel_value = prediction + compressed_array[idx]
            pixel_value = np.clip(pixel_value, 0, 255)
            image_array[i, j] = pixel_value
            idx += 1
    
    return image_array


In [3]:
class Node:
    def __init__(self, freq, symbol, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right
    
    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(data):
    frequency = Counter(data)
    heap = [Node(freq, symbol) for symbol, freq in frequency.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(left.freq + right.freq, None, left, right)
        heapq.heappush(heap, merged)
    
    return heap[0]

def generate_codes(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}
    if node:
        if node.symbol is not None:
            codebook[node.symbol] = prefix
        generate_codes(node.left, prefix + "0", codebook)
        generate_codes(node.right, prefix + "1", codebook)
    return codebook

def huffman_encode(data):
    tree = build_huffman_tree(data)
    codes = generate_codes(tree)
    encoded_data = ''.join(codes[symbol] for symbol in data)
    return encoded_data, codes

def huffman_decode(encoded_data, codes):
    reverse_codes = {v: k for k, v in codes.items()}
    current_code = ''
    decoded = []
    for bit in encoded_data:
        current_code += bit
        if current_code in reverse_codes:
            decoded.append(reverse_codes[current_code])
            current_code = ''
    return np.array(decoded, dtype=np.int16)


In [ ]:
input_folder = "org_ecg_10sec_resize"
reconstructed_folder  = "after compress calic huffman"
os.makedirs(reconstructed_folder, exist_ok=True)

In [5]:
durations = []
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        image_path = os.path.join(input_folder, filename)
        start_time = time.time()

        # Step 1: CALIC compress
        compressed_array, original_shape = compress_ecg_image(image_path)

        # Step 2: Huffman encode
        encoded_data, codes = huffman_encode(compressed_array)

        # Step 3: Huffman decode
        decoded_errors = huffman_decode(encoded_data, codes)

        # Step 4: CALIC decompress
        reconstructed_image = decompress_ecg_image(decoded_errors, original_shape)

        # Step 5: Save reconstructed image
        save_path = os.path.join(reconstructed_folder, filename)
        Image.fromarray(reconstructed_image).save(save_path)

        end_time = time.time()
        duration = end_time - start_time
        durations.append((filename, duration))
        print(f"{filename}: Processing time = {end_time - start_time:.4f} seconds")

csv_path = 'compress_duration_calic_huffman.csv'
with open(csv_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Image Name', 'Duration (seconds)'])
    writer.writerows(durations)

print(f"✅ Compression complete. Duration saved to: {csv_path}")

C:\Users\Jiawei\AppData\Local\Temp\ipykernel_16152\2698308395.py:16: RuntimeWarning: overflow encountered in scalar add
  prediction = (left + top) // 2
C:\Users\Jiawei\AppData\Local\Temp\ipykernel_16152\2698308395.py:38: RuntimeWarning: overflow encountered in scalar add
  prediction = (left + top) // 2


ecg_record_100.png: Processing time = 2.7331 seconds
ecg_record_101.png: Processing time = 2.7044 seconds
ecg_record_102.png: Processing time = 2.7063 seconds
ecg_record_103.png: Processing time = 2.7107 seconds
ecg_record_104.png: Processing time = 2.7185 seconds
ecg_record_105.png: Processing time = 2.7038 seconds
ecg_record_106.png: Processing time = 2.6990 seconds
ecg_record_107.png: Processing time = 2.7196 seconds
ecg_record_108.png: Processing time = 2.7027 seconds
ecg_record_109.png: Processing time = 2.7050 seconds
ecg_record_111.png: Processing time = 2.7032 seconds
ecg_record_112.png: Processing time = 2.7071 seconds
ecg_record_113.png: Processing time = 2.6994 seconds
ecg_record_114.png: Processing time = 2.6936 seconds
ecg_record_115.png: Processing time = 2.7044 seconds
ecg_record_116.png: Processing time = 2.7077 seconds
ecg_record_117.png: Processing time = 2.7144 seconds
ecg_record_118.png: Processing time = 2.7073 seconds
ecg_record_119.png: Processing time = 2.7038 s